In [ ]:
%matplotlib ipympl
import matplotlib as mpl
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np

import experimental.beacon_sim.ekf_slam_python as esp
import planning.probabilistic_road_map_python as prmp
from experimental.beacon_sim.ekf_slam_estimate_pb2 import EkfSlamEstimate as EstimateProto

import itertools
import embag_python as embag
import enum
from typing import NamedTuple


mpl.style.use('ggplot')

In [ ]:
bag_file_dir = Path('/home/erick/Dropbox (MIT)/brm_bag_files/gt_estimator')
raw_bag_file_dir = Path('/home/erick/Dropbox (MIT)/brm_bag_files/raw')
road_map_path = Path('/home/erick/Dropbox (MIT)/brm_bag_files/killian_court_road_map.pb')

In [ ]:
baseline_bags = sorted(list(bag_file_dir.glob('*baseline*present*[0-9].bag')))
brule_bags = sorted(list(bag_file_dir.glob('*brule*present*[0-9].bag')))
ONE_HUNDRED_MB = 100 * 1024 * 1024
brule_bags = [p for p in brule_bags if p.stat().st_size > ONE_HUNDRED_MB]

In [ ]:
brule_bags

In [ ]:
baseline_bags

In [ ]:
road_map = prmp.RoadMap.from_proto_string(road_map_path.read_bytes())

In [ ]:
def read_gt_bag(path: Path):
    bag = embag.Bag(path)
    map_from_robot_at_time = []
    for topic, msg, time in bag.read_messages(topics=["/map"]):
        proto_string = bytes(msg.estimate_proto)
        estimate = esp.EkfSlamEstimate.from_proto_string(proto_string)
        map_from_robot_at_time.append((estimate.time_of_validity, estimate.local_from_robot()))
    pos = np.stack([mfr.translation() for _, mfr in map_from_robot_at_time])
    return pos



In [ ]:
baseline_positions = [read_gt_bag(str(x)) for x in baseline_bags]
brule_positions = [read_gt_bag(str(x)) for x in brule_bags]

In [ ]:
def plot_road_map(road_map):
    pts = np.stack(road_map.points())
    plt.plot(pts[:, 0], pts[:, 1], '*')
    for i, pt in enumerate(pts):
        label = str(i)
        if i == 16:
            label = 'Start'
        elif i == 17:
            label = "Goal"
        plt.text(*pt, label)

    if road_map.has_start_goal():
        for idx, marker in [(road_map.START_IDX, 'rs'), (road_map.GOAL_IDX, 'y*')]:
            pt = road_map.point(idx)
            plt.plot(*pt, marker)
            

    line_segments = []
    for i in range(len(pts)):
        for j in range(i+1, len(pts)):
            if road_map.adj()[i, j] != 0:
                line_segments.append([pts[i, :], pts[j,:]])

    if road_map.has_start_goal():
        for idx in [road_map.START_IDX, road_map.GOAL_IDX]:
            pt = road_map.point(idx)
            neighbors = road_map.neighbors(idx)
            for _, neighbor_pt in neighbors:
                line_segments.append([pt, neighbor_pt])
    
    edges = mpl.collections.LineCollection(line_segments, colors=(0.6, 0.8, 0.6, 1.0))
    ax = plt.gca()
    ax.add_collection(edges)

In [ ]:

plt.figure()
plot_road_map(road_map)
for pos in baseline_positions:
    plt.plot(pos[:, 0], pos[:, 1])


plt.xlabel('X (m)')
plt.ylabel('Y (m)')
plt.title('Baseline Trajectories')
plt.axis('equal')

In [ ]:

plt.figure()
plot_road_map(road_map)
for pos in brule_positions:
    plt.plot(pos[:, 0], pos[:, 1])

plt.xlabel('X (m)')
plt.ylabel('Y (m)')
plt.title('BRULE Trajectories')
plt.axis('equal')

In [ ]:

class PointSource(enum.Enum):
    true = 1
    nominal = 2

class Point(NamedTuple):
    idx: int
    source: PointSource

def sample_path_edge(road_map, start_idx, end_idx, max_ds_m = 1.0):
    start_pos = road_map.point(start_idx)
    end_pos = road_map.point(end_idx)
    d_pos = end_pos - start_pos
    edge_length_m = np.linalg.norm(d_pos)
    num_steps = int((edge_length_m + 0.5 * max_ds_m) / max_ds_m)

    positions = []
    distances = []
    for i in range(0, num_steps+1):
        frac = i / num_steps
        positions.append(start_pos + frac * d_pos)
        distances.append(frac * edge_length_m)
    return positions, distances
        

def sample_node_path(road_map, node_path, max_ds_m = 1.0):
    positions = []
    distances = []
    for start, end in zip(node_path[:-1], node_path[1:]):
        edge_positions, edge_distances = sample_path_edge(road_map, start, end, max_ds_m)
        if not positions:
            positions.extend(edge_positions)
            distances.extend(edge_distances)
        else:
            positions.extend(edge_positions[1:])
            offset_dist_m = distances[-1]
            distances.extend([d + offset_dist_m for d in edge_distances[1:]])
    return np.vstack(positions), np.array(distances)

def find_nearest(q, polyline):
    delta = polyline - q
    norms = np.linalg.norm(delta, axis=1)
    return np.argmin(norms, keepdims=True)
    

def find_correspondence(positions, nominal_positions):
    D = {}
    unmatched_nom_idxs = set(range(len(nominal_positions)))
    for true_idx, true_pt in enumerate(positions):
        nearest_on_nominal_idxs = find_nearest(true_pt, nominal_positions)
        for nom_idx in nearest_on_nominal_idxs:
            nom_key = Point(nom_idx, PointSource.nominal)
            nearest_pts = D.get(nom_key, [])
            nearest_pts.append(Point(true_idx, PointSource.true))
            D[nom_key] = nearest_pts
            unmatched_nom_idxs -= {nom_idx}
        
    for nom_idx in unmatched_nom_idxs:
        nom_key = Point(nom_idx, PointSource.nominal)
        nom_pt = nominal_positions[nom_idx]
        true_idxs = find_nearest(nom_pt, positions)
        for true_idx in true_idxs:
            true_key = Point(true_idx, PointSource.true)
            if true_key in D:
                D[true_key].append(nom_key)
            else:
                nom_closest_to_true_idx = find_nearest(positions[true_idx], nominal_positions)
                nom_closest_to_true_key = Point(nom_closest_to_true_idx, PointSource.nominal)
                if nom_closest_to_true_key in D:
                    D[nom_closest_to_true_key].remove(true_key)
                    D[true_key] = [nom_idx]
                else:
                    del D[nom_closest_to_true_key]
                    D[true_key] = [nom_idx, nom_closest_to_true_idx]

    partition = []
    for true_idx in range(len(positions)):
        true_key = Point(true_idx, PointSource.true)
        if true_key in D:
            partition.append(([true_idx], D[key]))
    for nom_idx in range(len(nominal_positions)):
        nom_key = Point(nom_idx, PointSource.nominal)
        if nom_key in D:
            partition.append((D[nom_key], [nom_key]))
    return partition

def compute_minimal_error(positions, nominal_positions, correspondence):
    ...

def compute_crosstrack_error(positions, road_map, node_path):
    # Sample the node path 
    nominal_positions, geodesic_distances = sample_node_path(road_map, node_path)
    
    # Find a correspondence between the positions and the sampled positions
    correspondence = find_correspondence(positions, nominal_positions)

    # For each point on the nominal path, compute the minimum error
    compute_minimal_error(positions, nominal_positions, correspondence)
    
    # Compute the position error
    return nominal_positions, geodesic_distances, correspondence
    ...

In [ ]:
baseline_path = [16, 0, 5, 6, 10, 15, 17]
nominal_positions, geodesic_distances, correspondence = compute_crosstrack_error(baseline_positions[0], road_map, baseline_path)



In [ ]:
plt.figure()
plt.plot(baseline_positions[0][:,0], baseline_positions[0][:,1], label='true')
plt.plot(nominal_positions[:,0], nominal_positions[:,1], 'o-', label='nominal')

line_segments = []
for true_pts, nominal_pts in correspondence:
    for true_pt, nominal_pt in itertools.product(true_pts, nominal_pts):
        line_segments.append([baseline_positions[0][true_pt.idx, :], nominal_positions[nominal_pt.idx, :]])

edges = mpl.collections.LineCollection(line_segments, colors=(0.6, 0.8, 0.6, 1.0))
ax = plt.gca()
ax.add_collection(edges)

plt.legend()

In [ ]:
(true_pts, nominal_pts) = correspondence[0]

In [ ]:
len(true_pts[0])